# Caller

`Caller` is a class that makes it easy to call code in different threads/tasks.

## Threads


[Threads are not currently supported in Pyodide](https://github.com/pyodide/pyodide/issues/237), however a new caller instance is created emulating the functionality needed for thread related calls.

## Usage by the kernel

The kernel uses two `Caller` instances; one each for `shell` and `control`. 

In [ ]:
# A magic method provided by async kernel
%callers

In [ ]:
from async_kernel import Caller

Caller()

## Example
**This example requires ipywidgets!**

In [ ]:
import random
import time
import anyio

import ipywidgets as ipw

outputs = {}

stopper = ipw.RadioButtons(options=['Stop'], value=None)
display(stopper)

async def my_func(n):
    caller = Caller()
    if not (out := outputs.get(caller)):
        outputs[caller] = out = ipw.HTML(description=str(caller))
        out.style.description_width = "220px"
        display(out)
    sleep_time = random.random() / 4
    out.value = f"{n=:04d} sleeping {sleep_time * 1000:03.0f} ms"
    await anyio.sleep(sleep_time)
    return n


async def run_forever():
    n = 0
    while not stopper.value:
        n += 1
        yield Caller().to_thread(my_func, n)


async for fut in Caller().as_completed(run_forever()):
    result = await fut
    print(f"Finished: {result}", end="\r")



In [ ]:
# task
%callers